### Rasterize Pipeline Data
Extracts a subset of pipeline data and converts to raster format.  

In [2]:
#Imports
import arcpy, os

#Set environments
arcpy.env.workspace = '..\\scratch'
arcpy.env.scratchWorkspace = '..\\scratch'
arcpy.env.overwriteOutput = True

In [3]:
#Read in the pipeline data
gdb_NG = '\\\\ns-leviathan.win.duke.edu\\biogas_GIS\\Rextag_Zone2\\Rextag_Zone2_Natural_Gas.gdb\\Natural_Gas'
fc_NaturalGasPipelines =  os.path.join(gdb_NG,'NaturalGasPipelines')
if not arcpy.Exists(fc_NaturalGasPipelines):
    print("Error finding pipeline dataset!")
else:
    sr = arcpy.Describe(fc_NaturalGasPipelines).spatialReference
    srName = sr.name
    srCode = sr.GCSCode
    arcpy.env.outputCoordinateSystem=sr
    print("Natural gas pipeline feature class found.")
    print("Spatial reference set to {} (code:{})".format(srName,srCode))

Natural gas pipeline feature class found.
Spatial reference set to GCS_North_American_1983 (code:4269)


In [20]:
#Isolate NC Pipelines
theState = "North Carolina"
fc_StatePipelines = arcpy.Select_analysis(in_features=fc_NaturalGasPipelines,
                                          out_feature_class='statePipes.shp',
                                          where_clause="STATE_NAME = '{}'".format(theState)).getOutput(0)

In [18]:
outSR = arcpy.SpatialReference(26917)

In [ ]:
#Project to UTM Zone 17
fc_StatePipelines_prj = arcpy.management.Project(in_dataset=fc_StatePipelines,
                                                 out_dataset='../scratch/statePipes_prj.shp',
                                                 out_coor_system = outSR
                                                ).getOutput(0)

In [28]:
fc_StatePipelines_prj

'..\\scratch\\statePipes_prj.shp'

In [31]:
#Convert to 10 m raster
arcpy.env.extent = fc_StatePipelines_prj
arcpy.env.outputCoordinateSystem=fc_StatePipelines_prj
raster_StatePipelines = arcpy.conversion.PolylineToRaster(in_features=fc_StatePipelines_prj,
                                                          value_field='FID',
                                                          out_rasterdataset='../scratch/StatePipes2.img',
                                                          cell_assignment='MAXIMUM_LENGTH',
                                                          priority_field='NONE',
                                                          cellsize=10).getOutput(0)